In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.master("local").appName("Examples")\
.config("tech.sourced.api.repositories.path","/repositories")\
.getOrCreate()

In [2]:
repositoriesDf = spark.read.format("tech.sourced.api").option("table","repositories").load("/repositories")
referencesDf = spark.read.format("tech.sourced.api").option("table","references").load("/repositories")
commitsDf = spark.read.format("tech.sourced.api").option("table","commits").load("/repositories")
#TODO filesDf = spark.read.format("tech.sourced.api").option("table","files").load("/path/to/repositories")

### Get all HEAD references for all the repositories

In [3]:
headsDf = referencesDf.filter(referencesDf.name == 'refs/heads/HEAD')

### Get repositories that are marked as no forks

In [4]:
noForksDf = repositoriesDf.filter(repositoriesDf.is_fork == 0)

### Get only references for original repositories

In [5]:
noForksHeadsDf = headsDf.join(noForksDf, headsDf.repository_id == noForksDf.id).drop('repository_id').distinct()

In [6]:
headCommitsDf = noForksHeadsDf.join(commitsDf, noForksHeadsDf.hash == commitsDf.hash)

In [ ]:
noForksHeadsDf.show()

+---------------+--------------------+--------------------+--------------------+-------+
|           name|                hash|                  id|                urls|is_fork|
+---------------+--------------------+--------------------+--------------------+-------+
|refs/heads/HEAD|65f60c9cbd4d562f0...|github.com/joseph...|[https://github.c...|  false|
|refs/heads/HEAD|20779731d3185af5c...|github.com/nvbn/t...|[https://github.c...|  false|
|refs/heads/HEAD|6574f05356c681dd5...|github.com/isocpp...|[https://github.c...|  false|
|refs/heads/HEAD|c0b5382124e155249...|github.com/minima...|[https://github.c...|  false|
|refs/heads/HEAD|cff5fb452953800ed...|github.com/soimor...|[https://github.c...|  false|
|refs/heads/HEAD|34c43f4775971ab9b...|github.com/tornad...|[https://github.c...|  false|
|refs/heads/HEAD|7f9c998a03a91fa93...|github.com/donnem...|[https://github.c...|  false|
|refs/heads/HEAD|218b44996d5b51f95...|github.com/XX-net...|[https://github.c...|  false|
|refs/heads/HEAD|dfe6

In [ ]:
headCommitsDf.drop("blobs").drop("files").show()